In [12]:
# Gravitational Lensing Research
# Program to take a qlens-generated sample chain, derive kappa vs. radius values. No plotting.
# v5: Only DM.
# Switching to kpc rather than arc seconds.
# Also calculate the mass enclosed versus radius.
# Also calculate the radius containing 1% of the M200 mass.
# Saves the equal weighted chain.
# This version is for RXCJ2248

In [13]:
import time
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import timeit
from IPython.core.debugger import set_trace
import datetime
import concurrent.futures as cf
import multiprocessing as mp
import copy
from scipy import interpolate
from scipy.optimize import minimize_scalar
import pandas as pd
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy import constants as const

In [14]:
# for parallel processing
nproc = 1

##  Changing the output radii values to kpc. Make sure redshift is correct.

In [15]:
name = 'RXCJ2248'
dirname = 'RXCJ2248'
suffix = '.cnfw.v10c'
label = name + suffix
# Set your path here, to the "chains_..." directory of interest
#path = '/home/grant/Documents/qlens-beta/' + name + '/chains_' + label
# path = '/home/kea/KEVIN/UCI/Research/gravlensing/' + dirname + '/chains_' + label
path = '/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/' + dirname + '/chains_' + label
os.chdir(path)
print(os.getcwd())

/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/RXCJ2248/chains_RXCJ2248.cnfw.v10c


In [16]:
rs_dic = {'A611': 0.288, 'A2537': 0.294, 'RXCJ2248': 0.348, 'MS2137': 0.314, 'A383': 0.189,
          'A2261': 0.225, 'M2129': 0.589, 'A1703': 0.280,  'M1720': 0.387}

zlens = rs_dic[name]
print('zlens = ', zlens)

zlens =  0.348


In [17]:

cosmo = FlatLambdaCDM(H0=70 * u.km / u.s /u.Mpc, Om0=0.3)
print(cosmo)
rho_crit = cosmo.critical_density(zlens).to(u.M_sun / u.kpc**3)
print(rho_crit)
kpc_per_arcsec = cosmo.kpc_proper_per_arcmin(zlens) * u.arcmin / (60. * u.arcsec)
print (kpc_per_arcsec)
rho_200 = rho_crit * 200 * kpc_per_arcsec**3  # units of M_sun arcsec^-3
print('rho_200 = ', rho_200)
# get rid of units now
kpc_per_arcsec = kpc_per_arcsec.value
rho_200 = rho_200.value

FlatLambdaCDM(H0=70 km / (Mpc s), Om0=0.3, Tcmb0=0 K, Neff=3.04, m_nu=None, Ob0=None)
195.12769323056327 solMass / kpc3
4.92139759498602 kpc / arcsec
rho_200 =  4651727.483682498 solMass / arcsec3


In [18]:
# Calculate Sigma_crit, assuming a source redshift of 2.0.
sigma_crit = (const.c**2 * cosmo.angular_diameter_distance(2.0) / (4. * np.pi * const.G *\
                           cosmo.angular_diameter_distance(zlens) * cosmo.angular_diameter_distance_z1z2(zlens, 2.0))\
             ).to(u.M_sun / u.kpc**2)
print(sigma_crit)   
# get rid of units now
sigma_crit = sigma_crit.value

2226281196.5328746 solMass / kpc2


# Review the starting and stopping values!

In [19]:
# radii values, in KPC, log spaced. Based on 0.5 * innermost and 2.0 * outermost data points.
rstart = 17.12 * 0.5     
rstop  = 247.11 * 2.0
nr = 12  
radii_kpc = np.logspace(np.log10(rstart), np.log10(rstop), nr, endpoint=True)
radii_as = radii_kpc / kpc_per_arcsec
# QLens uses no endpoint, so it's stopping value needs to be calculated differently
ql_stop_as = radii_kpc[-1] * radii_kpc[-1] / radii_kpc[-2] / kpc_per_arcsec
radii_kpc = np.round(radii_kpc, 2)
print(f'Covers radii of {radii_kpc[0]:6.2f} kpc to    {radii_kpc[-1]:6.2f} kpc.')
print(f'Covers radii of {radii_as[0]:6.2f} arcsec to {radii_as[-1]:6.2f} arcsec.')
print(f'QLens stopping value is {ql_stop_as:6.2f} arc seconds')

Covers radii of   8.56 kpc to    494.22 kpc.
Covers radii of   1.74 arcsec to 100.42 arcsec.
QLens stopping value is 145.20 arc seconds


In [20]:
assert np.allclose(radii_as, np.logspace(np.log10(radii_kpc[0] / kpc_per_arcsec), np.log10(ql_stop_as),
                                         nr, endpoint=False), atol=0.01, rtol=0.001)
radii_as = np.round(radii_as, 2)
radii_kpc = np.round(radii_kpc, 2)

In [21]:
# now calculated the median and +/- 1-sigma values, accounting for sample weights.
def find_bands(p, samples, n_rad, weighted=True):
    # 'p' is the offset of the first column to sort by.
    # 'n_rad' is the number of columns to sort.
    # The first column is the weight, unless weighted=False.
    if not weighted:
        p += 1
        samples = np.hstack((np.ones((samples.shape[0], 1)), samples))
    total = [np.sum(samples[:, i + p] * samples[:, 0]) for i in range(n_rad)]
    sigma_minus  = np.zeros(n_rad)
    sigma_plus = np.zeros(n_rad)
    median = np.zeros(n_rad)
    for i in range(n_rad):
        runsum = 0.
        sm_flag = sp_flag = m_flag = False
        # sort the table by increasing kappa for the radius in question
        samsort =  samples[samples[:, i + p].argsort()]
        for row in range(len(samsort)):
            runsum += (samsort[row, 0] * samsort[row, i + p]) / total[i]
            if (not sm_flag) and (runsum > 0.158):
                sm_flag = True
                sigma_minus[i] = samsort[row, i + p]
            if (not sp_flag) and (runsum > 0.841):
                sp_flag = True
                sigma_plus[i] = samsort[row, i + p] 
            if (not m_flag) and (runsum > 0.5):
                m_flag = True
                median[i] = samsort[row, i + p]
    return sigma_minus, sigma_plus, median

In [22]:
# Open the paramnames file and display
with open(path + '/' + label + '.paramnames') as afile:
    paramnames = afile.readlines()
paramnames = [item[:-1] for item in paramnames]  # Strip off the \n character.
print(paramnames)

['mvir', 'c', 'rc_kpc', 'q', 'theta', 'xc', 'yc', 'shear1', 'shear2', 'mtot1', 'mtot2']


In [23]:
def ql_script5(i, p, params, tstart, ns):
    # assembles and runs the QLens script, returning ONE vector of kappa values (dmo)
    # Note that the first and pth columns are not parameters.
    # the pth column is the chi square.
    # Note that "like" is no longer on this list; equal weighted samples. 
    # Note that since we are now just looking at dark matter, this greatly simplifies the
    # script, eliminating parts b and c.
    # This could actually be done analytically.
    
    mvir, c, rc_kpc, q, theta, xc, yc, shear1, shear2, mtot1, mtot2, loglike = params[0:p]
    # NOTE: Check the above line; it may be subtly different for each halo, depending on the number of
    # perturbers, derived parameters, etc.
    
    pars0 = ' '.join([str(item) for item in [mvir, c, rc_kpc, q, theta, xc, yc]]) # Halo parameters
    
    pars1 = ' '.join([str(item) for item in [shear1, shear2]]) 

    # Note that since we are now just doing dark matter of the main halo, we don't need to enter 
    # the code for the BCG or perturbers!

    kappaname = 'plotdata/kappadist_temp_' + str(mp.current_process().name) + '.tmp'
    kappaname_dmo = 'plotdata/kappadist_temp_' + str(mp.current_process().name) + '.dmo.tmp'

    script_part_a1 = '''

    zlens=0.348    # REVIEW
    zsrc=2.0
    zsrc_ref=2.0
    shear_components on
    major_axis_along_y off    

    sci_notation off
    gridtype cartesian  # REVIEW
    grid -70 65 -45 55  # REVIEW
    imgdata read ../RXCJ2248_SL_Data_v3.dat   #REVIEW
    '''

    script_part_a2 = '''
    lens clear
    lens cnfw pmode=3 '''
    # here we insert the parameters for the first lens (halo)

    script_part_b1 =''' shear='''
    # here we insert the paramters for the shear
    
    script_part_b2 ='''
    lens pjaffe '''
    # here we insert the parameters for the second lens (BCG)

    script_part_b3 ='''
    lens pjaffe '''
    # pert 1

    script_part_b4 ='''
    lens pjaffe  '''
    # pert 2
    
    script_part_b5 ='''
    lens pjaffe  '''
    # pert 3
    
#     script_part_c = '''
# lens perturber pjaffe	  0.275/anchor=5,0	0.275/anchor=5,1	0.0096/anchor=5,2    0.67	     80.6	-10.88	10.22	# 4

# lens perturber pjaffe	  0.209/anchor=5,0    	0.209/anchor=5,1	0.0073/anchor=5,2    0.84	     128.4	 -16.79	 0.60	# 5

# lens perturber pjaffe	  0.158/anchor=5,0    	0.158/anchor=5,1	0.0055/anchor=5,2	 0.9		 131.6	 1.13	 -2.78	# 6

# lens perturber pjaffe	  0.240/anchor=5,0    	0.240/anchor=5,1	0.0084/anchor=5,2	 0.79	      61.7	 -13.68	 12.87	# 7

#     '''
    # Other perturbers, if any, can go in part c above.
    
    script_part_d = '''
    plotkappa ''' + str(rstart / kpc_per_arcsec) + ' ' + str(ql_stop_as)\
    + ' ' + str(nr) + ' '

    script_part_e = '''
    cosmology
    '''
    
    script = script_part_a1 + script_part_a2 + pars0 \
     + script_part_d + kappaname_dmo + ' lens=0\n' \
     + script_part_e
    
    #         + script_part_b1 + pars1\
#         + script_part_b2 + pars2 + script_part_b3 + pars3  \
#         + script_part_b4 + pars4 + script_part_b5 + pars5  \
#         + script_part_c \
#         + script_part_d + kappaname + ' \n' \
    
    sc_name = 'plotdata/kappascript_' + str(mp.current_process().name) + '.in'
    
    # double check redshift
    rs_pos = script.find('zlens=')
    if rs_pos == -1:
        print("Can't locate 'zlens=' in script.")
        sys.exit()    
    eol = script.find('# REVIEW', rs_pos)
    assert float(script[rs_pos + 6: eol]) == zlens
    
    with open(sc_name, 'w') as outfile:
        outfile.write(script)

    # Now, run the script, saving the output in a temporary file
    os.system('qlens ' + sc_name + ' -q | tee plotdata/scriptout' + str(mp.current_process().name) + '.tmp')
    
    # read back the cosmology and determine the r_200 of lens 0
    with open('plotdata/scriptout' + str(mp.current_process().name) + '.tmp', 'r') as outf:
        output = ''.join(outf.readlines())
    cos_pos = output.find('cosmology')
    lens0_pos = output.find('Lens 0:', cos_pos)
    r200_pos = output.find('r_200 = ', lens0_pos)
    kpc_pos = output.find('kpc', r200_pos)
    r200 = np.float(output[r200_pos + 7 : kpc_pos])
    m200 = 4. * np.pi / 3. * r200**3 * rho_200 / kpc_per_arcsec**3 
    
    # read in kappa file back in, and store results
#     kdat = np.loadtxt(kappaname)
#     kappa = kdat[:, 1]
    kdat_dmo = np.loadtxt(kappaname_dmo)
    kappa_dmo = kdat_dmo[:, 1]
    encl_mass = kdat_dmo[:, 4]
    # The above assumes that the QLens data is in M_sun.
#     kappa_bary = kappa - kappa_dmo
    
    # Calculate the radius containing 1% of m200:
    f_encl = interpolate.interp1d(radii_kpc, encl_mass, fill_value='extrapolate')
    r_1_res = minimize_scalar(lambda r: np.abs(f_encl(r) - 0.01 * m200))
    if r_1_res.success != True:
        set_trace()
    r_1pct = r_1_res.x
    if r_1pct <0. or r_1pct > 4000.:
        set_trace()
    
    if i==0:
        assert np.all(np.abs((kdat_dmo[:, 0] * kpc_per_arcsec - radii_kpc) / radii_kpc) \
                      < .01)
    if i%50 == 1:
        tavg = (timeit.default_timer() - tstart) / (i+1.)
        proj_fin =  tavg * (ns -i -1)
        sys.stdout.write("\r{0:5.2%} complete. Avg time per iteration: {1:8.6f} sec. Projected finish in {2:4.2f} min. "\
                   .format((float(i) / ns), tavg,  proj_fin/60))
    sys.stdout.flush() # required in multiprocessing or else it buffers this output
    return kappa_dmo, encl_mass, r_1pct

In [24]:
# Now doing this calculation not in a function.
# Import the chain. This is the EQUAL weighted chain.
samples = np.loadtxt(path + '/' + label + 'post_equal_weights.dat', comments="#", delimiter=None, unpack=False)
samples = np.array(samples)

In [25]:
# Load the ranges, and scale the parameters.
ranges = np.loadtxt(path + '/' + label + '.ranges', comments="#", delimiter=None, unpack=False)
for i, (lwr, upr) in enumerate(ranges):  # All columns but the last one in samples.
    if upr != 1.e30:
        samples[:, i] = samples[:, i] * (upr - lwr) + lwr

In [26]:
ns, p = samples.shape
print(samples.shape)
print('Median parameter values: ')
print([f'{pn}: {mv:.4g}' for (pn, mv) in zip(paramnames, [np.median(i) for i in samples.T])])

chain_columns = paramnames + ['loglike']

# for testing only, you can uncomment the two lines below, and it will only use the last 500 samples,
# which is useful to quickly see if things are working right.

# ns = 500
# samples = samples[-ns:]

(40878, 12)
Median parameter values: 
['mvir: 1.839e+15', 'c: 5.39', 'rc_kpc: 19.82', 'q: 0.5044', 'theta: -35.9', 'xc: -0.5391', 'yc: 0.3222', 'shear1: 0.005871', 'shear2: 0.02715', 'mtot1: 8.159e+11', 'mtot2: 7.412e+09']


In [27]:
# make sure there is a directory called plotdata. If not, create it.
if not os.path.exists(os.getcwd()+'/plotdata'):
    os.makedirs(os.getcwd()+'/plotdata')

results = []
tstart = timeit.default_timer()
with cf.ProcessPoolExecutor(nproc) as pp:
    for i, params in enumerate(samples[:, 0:p]):
        results.append((i, pp.submit(ql_script5, i, p, params, tstart, ns)))

# code waits here until all processes are finished
sys.stdout.write('\r                                                                                      ')
print("\nKappa calculation completed in ", np.round((timeit.default_timer() - tstart) / 60, 2), " minutes.")

                                                                                      
Kappa calculation completed in  14.73  minutes.


In [28]:
all_names = []  #['all_%07.2f' % rad for rad in radii_kpc]
dm_names = ['dm_%07.2f' % rad for rad in radii_kpc]
bary_names = []  #['bary_%07.2f' % rad for rad in radii_kpc]
m_names = ['m_encl_%07.2f' % rad for rad in radii_kpc]
col_names = all_names + dm_names + bary_names + m_names + ['r_1pct']

# Use a numpy array for holding everything. DataFrames are too slow.
samples2 = np.append(samples, np.full((ns, nr * 2 + 1), np.nan), axis=1)  # Note: changed *4 to *3 here

old_i = -1
print('Updating data array...')
for (i,k) in results:
    k_dm, m_encl, r_1= k.result()
    reslist = list(k_dm) + list(m_encl)+ [r_1]
    samples2[i, p:] = reslist
    if old_i +1 != i:
        print('WARNING: Out of order at i=', i)
    old_i = i
print('Data updated.')

Updating data array...
Data updated.


In [29]:
print('Making bands...')
# sm, sp, med = find_bands(0, samples2[:, p: p + nr], nr, weighted=False)
sm_dmo, sp_dmo, med_dmo = find_bands(0, samples2[:, p + nr * 0: p + nr * 1], nr, weighted=False)
# sm_bary, sp_bary, med_bary = find_bands(0, samples2[:, p + nr * 2: p + nr *3], nr, weighted=False)
sm_mencl, sp_mencl, med_mencl = find_bands(0, samples2[:, p + nr * 1: p + nr * 2], nr, weighted=False)
sm_r1, sp_r1, med_r1 = find_bands(0, samples2[:, p + nr * 2: p + nr * 2 + 1], 1, weighted=False)
print("Bands made.")

Making bands...
Bands made.


In [30]:
# Multiply the kappas in the samples2 array by Sigma_crit, to yield surface density in M_sun kpc^-2
samples3 = samples2
samples3[:, p: p + nr] = samples2[:, p: p + nr] * sigma_crit  # Note, this is altered for dmo, otherwise need nr*3
df = pd.DataFrame(samples3, columns = chain_columns + col_names)
df.describe()

,mvir,c,rc_kpc,q,theta,xc,yc,shear1,shear2,mtot1,...,m_encl_0025.87,m_encl_0037.41,m_encl_0054.09,m_encl_0078.21,m_encl_0113.08,m_encl_0163.50,m_encl_0236.41,m_encl_0341.81,m_encl_0494.22,r_1pct
count,4.087800e+04,40878.000000,40878.000000,40878.000000,40878.000000,40878.000000,40878.000000,40878.000000,40878.000000,4.087800e+04,...,4.087800e+04,4.087800e+04,4.087800e+04,4.087800e+04,4.087800e+04,4.087800e+04,4.087800e+04,4.087800e+04,4.087800e+04,40878.000000
mean,1.848519e+15,5.449458,21.537840,0.504281,-35.889833,-0.545280,0.324391,0.005858,0.027077,8.950429e+11,...,7.416650e+12,1.449281e+13,2.768320e+13,5.145163e+13,9.260865e+13,1.606802e+14,2.676117e+14,4.264283e+14,6.488860e+14,42.464579
std,1.408857e+14,0.654340,11.393482,0.013269,0.410105,0.208035,0.164816,0.004859,0.012385,7.122293e+11,...,3.775497e+11,5.520848e+11,7.281558e+11,8.549323e+11,9.585740e+11,1.479274e+12,3.382544e+12,8.020427e+12,1.755874e+13,1.592729
min,1.418113e+15,3.707292,0.001222,0.447728,-37.510151,-1.444772,-0.329024,-0.019284,-0.022612,1.000578e+10,...,6.385700e+12,1.283326e+13,2.525078e+13,4.828014e+13,8.899978e+13,1.547543e+14,2.539628e+14,3.952640e+14,5.837414e+14,37.067278
25%,1.748689e+15,4.976196,13.104794,0.495481,-36.166841,-0.683223,0.213828,0.002598,0.018746,2.023390e+11,...,7.130122e+12,1.407233e+13,2.713318e+13,5.081082e+13,9.191570e+13,1.596806e+14,2.653463e+14,4.210450e+14,6.369294e+14,41.364692
50%,1.839097e+15,5.390324,19.824124,0.504414,-35.899245,-0.539149,0.322193,0.005871,0.027146,8.158775e+11,...,7.393340e+12,1.447783e+13,2.768957e+13,5.148628e+13,9.264584e+13,1.607010e+14,2.676292e+14,4.264279e+14,6.486312e+14,42.390886
75%,1.937086e+15,5.865426,28.383153,0.513305,-35.615958,-0.402344,0.434084,0.009110,0.035447,1.453420e+12,...,7.673426e+12,1.488529e+13,2.822511e+13,5.211160e+13,9.333716e+13,1.617046e+14,2.699096e+14,4.317851e+14,6.604379e+14,43.486476
max,2.464483e+15,9.418072,91.256955,0.559011,-34.032077,0.216247,1.002830,0.030492,0.077162,3.386619e+12,...,8.913909e+12,1.651625e+13,3.013513e+13,5.417483e+13,9.554669e+13,1.658738e+14,2.811612e+14,4.605071e+14,7.231346e+14,50.183709


In [41]:
med_dmo

array([1.711134 , 1.652322 , 1.570038 , 1.460791 , 1.324439 , 1.163165 ,
       0.9849361, 0.7998115, 0.6186204, 0.4532854, 0.3133661, 0.204264 ])

In [ ]:
# The values above should start at of approx. 1.0 to 2.0, then decrease gradually to approx. 0.1 to 0.3.

In [31]:
# np.savez(label + '.all.bands5', radii_kpc, sm, sp, med)
np.savez(label + '.dmo.bands5', radii_kpc, sm_dmo, sp_dmo, med_dmo)
# np.savez(label + '.bary.bands5', radii_kpc, sm_bary, sp_bary, med_bary)
np.savez(label + '.mencl.bands5', radii_kpc, sm_mencl, sp_mencl, med_mencl)
np.savez(label + '.r1pct.bands5', radii_kpc, sm_r1, sp_r1, med_r1)
with open(label + '.column_names', 'w') as colfile:
    colfile.writelines('\n'.join(list(df.columns.values)))
df.to_csv(label + '.csv')
print('Results saved to bands file, and dataframe saved to .csv file with label ', label)
print('Path: ', path)

Results saved to bands file, and dataframe saved to .csv file with label  RXCJ2248.cnfw.v10c
Path:  /Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/RXCJ2248/chains_RXCJ2248.cnfw.v10c


In [32]:
# Use Notebook "Kappa band plotter v5" to plot the results.

In [33]:
df.columns

Index(['mvir', 'c', 'rc_kpc', 'q', 'theta', 'xc', 'yc', 'shear1', 'shear2',
       'mtot1', 'mtot2', 'loglike', 'dm_0008.56', 'dm_0012.38', 'dm_0017.90',
       'dm_0025.87', 'dm_0037.41', 'dm_0054.09', 'dm_0078.21', 'dm_0113.08',
       'dm_0163.50', 'dm_0236.41', 'dm_0341.81', 'dm_0494.22',
       'm_encl_0008.56', 'm_encl_0012.38', 'm_encl_0017.90', 'm_encl_0025.87',
       'm_encl_0037.41', 'm_encl_0054.09', 'm_encl_0078.21', 'm_encl_0113.08',
       'm_encl_0163.50', 'm_encl_0236.41', 'm_encl_0341.81', 'm_encl_0494.22',
       'r_1pct'],
      dtype='object')

In [34]:
# Make a dataframe for Manoj to use in SIDM calculations.
dfsidm = df[['mvir', 'c', 'rc_kpc', 'mtot1', 'loglike',  *dm_names]]
dfsidm.rename({'mtot1': 'mBCG'}, axis='columns', inplace=True)

/Users/kevin/opt/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [35]:
means = dfsidm.mean(axis=0, numeric_only=True)
stdev = dfsidm.std(axis=0, numeric_only=True)
r, c = dfsidm.shape
r_thin_target = 5000.
stride = max(1, int(r / r_thin_target))
r_thin = int(r / stride)
print(f'Original dataframe is {r} rows. Attempting to thin to {r_thin}')

Original dataframe is 40878 rows. Attempting to thin to 5109


In [36]:
dfthin = dfsidm[::stride]
print(dfthin.shape)
thin_means = dfthin.mean(axis=0, numeric_only=True)
thin_stdev = dfthin.std(axis=0, numeric_only=True)
mean_chg = np.sqrt(np.mean(((means - thin_means) / means)**2))
sd_chg = np.sqrt(np.mean(((stdev - thin_stdev) / stdev)**2))
print(f'RMS mean change is {mean_chg:.1%}')
print(f'RMS stdev change is {sd_chg:.1%}')
assert mean_chg < 0.01
assert sd_chg < 0.02

(5110, 17)
RMS mean change is 0.3%
RMS stdev change is 1.0%


In [37]:
# save if error tolerance met
dfthin.to_csv(label + '.SIDM.csv')
print(f'Saved thinned SIDM dataframe to {label}.SIDM.csv')

Saved thinned SIDM dataframe to RXCJ2248.cnfw.v10c.SIDM.csv


In [38]:
dfthin.describe()

,mvir,c,rc_kpc,mBCG,loglike,dm_0008.56,dm_0012.38,dm_0017.90,dm_0025.87,dm_0037.41,dm_0054.09,dm_0078.21,dm_0113.08,dm_0163.50,dm_0236.41,dm_0341.81,dm_0494.22
count,5.110000e+03,5110.000000,5110.000000,5.110000e+03,5110.000000,5.110000e+03,5.110000e+03,5.110000e+03,5.110000e+03,5.110000e+03,5.110000e+03,5.110000e+03,5.110000e+03,5.110000e+03,5.110000e+03,5.110000e+03,5.110000e+03
mean,1.849007e+15,5.445033,21.496627,9.072186e+11,-42.760813,3.817064e+09,3.679471e+09,3.491438e+09,3.246454e+09,2.943129e+09,2.587198e+09,2.192480e+09,1.780261e+09,1.376748e+09,1.008605e+09,6.973203e+08,4.543723e+08
std,1.385936e+14,0.642713,11.234691,7.100991e+11,2.501500,2.741335e+08,2.175188e+08,1.589643e+08,1.039179e+08,5.842773e+07,2.812180e+07,1.750103e+07,1.887757e+07,2.159716e+07,2.421675e+07,2.503857e+07,2.284468e+07
min,1.468804e+15,3.796233,0.002746,1.001039e+10,-56.173201,3.217709e+09,3.169329e+09,3.089798e+09,2.949224e+09,2.747563e+09,2.478254e+09,2.129020e+09,1.706331e+09,1.297385e+09,9.277867e+08,6.184912e+08,3.855518e+08
25%,1.748949e+15,4.982282,13.319280,2.165970e+11,-44.235759,3.610724e+09,3.515567e+09,3.371268e+09,3.167213e+09,2.899294e+09,2.567398e+09,2.180883e+09,1.767772e+09,1.362375e+09,9.922644e+08,6.797724e+08,4.383164e+08
50%,1.842735e+15,5.382210,19.650624,8.429651e+11,-42.518356,3.788704e+09,3.665377e+09,3.488203e+09,3.248359e+09,2.946643e+09,2.588565e+09,2.192469e+09,1.780434e+09,1.377092e+09,1.009136e+09,6.972862e+08,4.542048e+08
75%,1.935888e+15,5.853338,28.033491,1.462913e+12,-41.040539,3.989747e+09,3.824820e+09,3.603987e+09,3.323439e+09,2.988215e+09,2.608130e+09,2.204327e+09,1.792831e+09,1.391188e+09,1.024988e+09,7.139524e+08,4.694942e+08
max,2.446971e+15,8.061289,74.583217,3.147034e+12,-36.130069,5.001336e+09,4.517091e+09,4.033487e+09,3.552219e+09,3.083537e+09,2.668554e+09,2.253402e+09,1.846411e+09,1.465729e+09,1.113716e+09,8.034876e+08,5.481138e+08


In [39]:
# Other statistics that should accompany the csv file:
#  Offset
# BCG scale radius and core radius

In [40]:
sys.exit()

SystemExit: 

/Users/kevin/opt/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# copy to the 'kappas' folder
os.chdir(path)
os.system('cp ' + label + '*.bands5.npz  "' + path + '/../../kappas/"')
os.system('cp ' + label + '.column_names  "' + path + '/../../kappas/"')
os.system('cp ' + label + '.csv  "' + path + '/../../kappas/"')